In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import nvtripy as tp


# -----------------------------
# 1. Модель LeNet на PyTorch
# -----------------------------
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.out = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.reshape(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.out(x)
        return x


# -----------------------------
# 2. Эквивалентная модель LeNet на TriPy
# -----------------------------
class TripyLeNet(tp.Module):
    def __init__(self, dtype=tp.float32):
        super().__init__()

        # те же размеры каналов и ядер, что и в PyTorch-версии
        self.conv1 = tp.Conv(
            in_channels=3,
            out_channels=6,
            kernel_dims=(5, 5),
            dtype=dtype,
        )
        self.conv2 = tp.Conv(
            in_channels=6,
            out_channels=16,
            kernel_dims=(5, 5),
            dtype=dtype,
        )

        self.fc1 = tp.Linear(16 * 5 * 5, 120, dtype=dtype)
        self.fc2 = tp.Linear(120, 84, dtype=dtype)
        self.out = tp.Linear(84, 10, dtype=dtype)

    def forward(self, x):
        # вход: (N, 3, 32, 32)
        x = self.conv1(x)
        x = tp.relu(x)
        x = tp.maxpool(x, kernel_dims=(2, 2), stride=(2, 2))

        x = self.conv2(x)
        x = tp.relu(x)
        x = tp.maxpool(x, kernel_dims=(2, 2), stride=(2, 2))

        # выпрямление фичей: (N, 16*5*5)
        x = tp.reshape(x, (x.shape[0], -1))

        x = self.fc1(x)
        x = tp.relu(x)

        x = self.fc2(x)
        x = tp.relu(x)

        x = self.out(x)
        return x


# -----------------------------
# 3. Конвертация state_dict PyTorch -> TriPy
# -----------------------------
def torch_state_to_tripy_state(torch_state_dict, dtype=tp.float32):
    """
    Преобразует state_dict из PyTorch в словарь TriPy-тензоров
    с теми же именами параметров.
    """
    tripy_state = {}
    for name, param in torch_state_dict.items():
        # на всякий случай уводим на CPU и в float32
        np_value = param.detach().cpu().numpy().astype("float32")
        tripy_state[name] = tp.Tensor(np_value, dtype=dtype)
    return tripy_state


# -----------------------------
# 4. Инициализация моделей и перенос весов
# -----------------------------
# загружаем обученную PyTorch-модель
pytorch_model = LeNet()
pytorch_model.load_state_dict(
    torch.load("lenet_cifar10.pth", map_location="cpu")
)
pytorch_model.eval()

# создаём TriPy-модель с идентичной архитектурой
tripy_model = TripyLeNet(dtype=tp.float32)

# переносим веса
tripy_state = torch_state_to_tripy_state(pytorch_model.state_dict())
missing_keys, unexpected_keys = tripy_model.load_state_dict(
    tripy_state, strict=True
)

print("Missing keys in TriPy model:", missing_keys)
print("Unexpected keys in TriPy model:", unexpected_keys)


# -----------------------------
# 5. Проверка эквивалентности выходов
# -----------------------------
def torch_to_tripy_tensor(x_torch, dtype=tp.float32):
    """
    Конвертация входного батча из PyTorch-тензора
    в TriPy-тензор.
    """
    np_x = x_torch.detach().cpu().numpy().astype("float32")
    return tp.Tensor(np_x, dtype=dtype)


# тестовый батч изображений CIFAR-10
x_torch = torch.randn(16, 3, 32, 32)  # вместо этого можно подставить реальные данные

with torch.no_grad():
    y_torch = pytorch_model(x_torch)

x_tripy = torch_to_tripy_tensor(x_torch)
y_tripy = tripy_model(x_tripy)

# конвертация выхода TriPy обратно в NumPy для сравнения
y_torch_np = y_torch.detach().cpu().numpy()
y_tripy_np = y_tripy.numpy()  # предполагается наличие метода .numpy()

max_abs_diff = np.max(np.abs(y_tripy_np - y_torch_np))
print(f"Максимальное абсолютное расхождение выходов: {max_abs_diff:.6e}")


torch.Size([1000, 10])
